In [25]:
import re

import pandas as pd
import numpy as np
from slugify import slugify

NUM_RE = re.compile(r'\d+')

In [28]:
def read_file(year):
    df = pd.read_csv('data/lv_2014_raw.csv', encoding='utf-16', delimiter=';', skiprows=3, names=['recipient_name',
                       'recipient_location',
                       'scheme',
                       'amount'])
    df['year'] = year
    return df

df = pd.concat([read_file(year) for year in range(2014, 2016)])
df.head()

,recipient_name,recipient_location,scheme,amount,year
0,Elzabeta Reine,Riebiņu nov.,2013-2014,4064.73,2014
1,NaN,NaN,Īpašais atbalsts par pienu,570.67,2014
2,NaN,NaN,Mazāklabvēlīgo apvidus kompensācija,773.30,2014
3,NaN,NaN,Vienotais platības maksājums,2386.92,2014
4,NaN,NaN,Mazāklabvēlīgo apvidus kompensācija,333.84,2014


In [29]:
df['recipient_postcode'] = None
df['recipient_address'] = None
df['recipient_country'] = 'LV'
df['currency'] = 'EUR'

In [30]:
df['recipient_id'] = df['recipient_name'].apply(lambda x: x if pd.notnull(x) and NUM_RE.match(x) else np.nan)
df['recipient_id'] = df.apply(lambda x: 'LV-%s-%s' % (x['year'], x['recipient_id']) if pd.notnull(x['recipient_id']) else
                              'LV-%s-%s' % (slugify(
                                      x['recipient_location'] if pd.notnull(x['recipient_location']) else ''
                                  ), slugify(
                                      x['recipient_name'] if pd.notnull(x['recipient_name']) else ''
                                  )
                                ), 1)
df['recipient_name'] = df['recipient_name'].apply(lambda x: np.nan if pd.isnull(x) or NUM_RE.match(x) else x)

In [31]:
df.head()

,recipient_name,recipient_location,scheme,amount,year,recipient_postcode,recipient_address,recipient_country,currency,recipient_id
0,Elzabeta Reine,Riebiņu nov.,2013-2014,4064.73,2014,None,None,LV,EUR,LV-Riebinu-nov-Elzabeta-Reine
1,NaN,NaN,Īpašais atbalsts par pienu,570.67,2014,None,None,LV,EUR,LV--
2,NaN,NaN,Mazāklabvēlīgo apvidus kompensācija,773.30,2014,None,None,LV,EUR,LV--
3,NaN,NaN,Vienotais platības maksājums,2386.92,2014,None,None,LV,EUR,LV--
4,NaN,NaN,Mazāklabvēlīgo apvidus kompensācija,333.84,2014,None,None,LV,EUR,LV--


In [32]:
df.to_csv('lv_2014_2015.csv.gz', index=False, compression='gzip', encoding='utf-8')